In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **MLflow Setup**

In [ ]:
# install mlflow
!pip install mlflow --quiet

In [ ]:
# install pyngrok to expose mlflow UI
!pip install pyngrok --quiet

In [ ]:
from pyngrok import ngrok
# Start MLflow UI server in background
get_ipython().system_raw("mlflow ui --port 5000 &")

# created a ngrok account and authenticated
ngrok.set_auth_token("30pLtpE03zO9a3mgzMsIeykbCNI_2N9HxvHKvSBt8tFtLec2W")

from pyngrok import ngrok
# kill any existing tunnels
ngrok.kill()
public_url = ngrok.connect(5000)
print(f"MLflow UI URL: {public_url.public_url}")

import mlflow
# Set tracking URI to public ngrok URL or localhost if running locally
mlflow.set_tracking_uri(public_url.public_url)  # or "http://localhost:5000" if no ngrok
mlflow.set_experiment("AutismDetection-Classifier")

# **Data Loading and Basic EDA**

In [1]:
# define the train directory
import os
path = '/kaggle/input/autism/autism_split'
train_dir = os.path.join(path,'train')
print(train_dir)

/kaggle/input/autism/autism_split/train


In [2]:
# count of the images in each folder of train and test
path = path
train_folder = os.path.join(path,'train')
train_a_folder = os.path.join(train_folder,'autistic')
train_na_folder = os.path.join(train_folder,'non_autistic')

val_folder = os.path.join(path,'val')
val_a_folder = os.path.join(val_folder,'autistic')
val_na_folder = os.path.join(val_folder,'non_autistic')

test_folder = os.path.join(path,'test')
test_a_folder = os.path.join(test_folder,'autistic')
test_na_folder = os.path.join(test_folder,'non_autistic')

# count train
train_a_count = len(os.listdir(train_a_folder))
train_na_count = len(os.listdir(train_na_folder))

# count test
val_a_count = len(os.listdir(val_a_folder))
val_na_count = len(os.listdir(val_na_folder))

# count train
test_a_count = len(os.listdir(test_a_folder))
test_na_count = len(os.listdir(test_na_folder))

# print train
print('---------------In TRAIN folder----------')
print("Number of autistic images:", train_a_count)
print("Number of non_autistic images:", train_na_count)
# print val
print('\n\n---------------In VAL folder----------')
print("Number of autistic images:", val_a_count)
print("Number of non_autistic images:", val_na_count)
# print test
print('\n\n---------------In TEST folder----------')
print("Number of autistic images:", test_a_count)
print("Number of non_autistic images:", test_na_count)

---------------In TRAIN folder----------
Number of autistic images: 2335
Number of non_autistic images: 2335


---------------In VAL folder----------
Number of autistic images: 584
Number of non_autistic images: 584


---------------In TEST folder----------
Number of autistic images: 12
Number of non_autistic images: 12


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

base_dir = path   # update this path

for split in ['train', 'val', 'test']:
    for cls in ['autistic', 'non_autistic']:
        cls_dir = os.path.join(base_dir, split, cls)
        img_files = [f for f in os.listdir(cls_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

        # Select up to 3 images to show as subplots
        plt.figure(figsize=(4,7))
        n_images = min(3, len(img_files))
        fig, axs = plt.subplots(1, n_images, figsize=(5 * n_images, 5))
        fig.suptitle(f"{split} - {cls}")

        for i in range(n_images):
            img_path = os.path.join(cls_dir, img_files[i])
            img = Image.open(img_path)
            axs[i].imshow(img)
            axs[i].set_title(img_files[i])
            axs[i].axis('off')

        plt.tight_layout()
        plt.show()


# **Data Generator for Model pipelining the data**

In [ ]:
train_dir

In [ ]:
# data flow from directory
import tensorflow as tf



# Load train data
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',        # for binary classification (0/1)
    image_size=(224,224),
    batch_size=32,
    shuffle=True
)

In [ ]:
# define the val directory
val_dir = os.path.join(path,'val')
val_dir

In [ ]:
# load the validation data

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='binary',        # for binary classification (0/1)
    image_size=(224,224),
    batch_size=32,
    shuffle=False
)

# MLflow tracking and logging function

In [ ]:
import mlflow
import mlflow.keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import tempfile
import os

def train_and_log_model(model, model_name, train_ds, val_ds, epochs=5, optimizer='adam'):
    with mlflow.start_run(run_name=model_name):

        # log model parameters
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("epochs", epochs)

        # compile the model
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        # train the model
        history = model.fit(train_ds,
                            validation_data=val_ds,
                            epochs=epochs)

        # log final metrics
        mlflow.log_metrics({
            "train_accuracy": history.history['accuracy'][-1],
            "val_accuracy": history.history['val_accuracy'][-1],
            "train_loss": history.history['loss'][-1],
            "val_loss": history.history['val_loss'][-1]
        })

        # Get predictions and labels for confusion matrix
        y_true = []
        y_pred = []
        for X_batch, y_batch in val_ds:
            preds = model.predict(X_batch,verbose=0)
            y_true.extend(y_batch.numpy())
            y_pred.extend(np.round(preds).flatten())

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        fig, ax = plt.subplots(figsize=(5, 5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Autistic', 'Autistic'], yticklabels=['Non-Autistic', 'Autistic'])
        plt.ylabel("Actual")
        plt.xlabel("Predicted")
        
        # Save confusion matrix image
        temp_dir = tempfile.mkdtemp()
        cm_path = os.path.join(temp_dir, "confusion_matrix.png")
        plt.savefig(cm_path)
        plt.close()
        mlflow.log_artifact(cm_path, artifact_path="plots")

        # Classification report
        clf_report = classification_report(y_true, y_pred, target_names=['Non-Autistic', 'Autistic'])
        report_path = os.path.join(temp_dir, "classification_report.txt")
        with open(report_path, "w") as f:
            f.write(clf_report)
        mlflow.log_artifact(report_path, artifact_path="reports")

        # Log model
        mlflow.keras.log_model(model, "model")
        print(f"Model: {model_name} and its metrics, confusion matrix, and report logged to MLflow.")


# **LeNet**

In [ ]:
# import packages
import tensorflow as tf
from tensorflow.keras import layers, models

# define the model
def build_lenet(input_shape=(224,224,3)):
  model = models.Sequential()

  # normalization of input images
  model.add(layers.Rescaling(1./255, input_shape=input_shape))

  # layer 1
  model.add(layers.Conv2D(6,kernel_size=(5,5),activation='relu',padding='same'))
  model.add(layers.AveragePooling2D(pool_size=(2,2)))

  # layer 2
  model.add(layers.Conv2D(16, kernel_size=(5,5),activation='relu'))
  model.add(layers.AveragePooling2D(pool_size=(2,2)))

  # layer 3
  model.add(layers.Flatten())
  model.add(layers.Dense(120,activation='relu'))

  # layer 4
  model.add(layers.Dense(64,activation='relu'))

  # layer 5
  model.add(layers.Dense(1,activation='sigmoid'))

  return model


In [ ]:
# train and track
# Set experiment ONCE at the start of your notebook/script
mlflow.set_experiment("Autism_Face_Classifier")

# call the model to build
lenet_model = build_lenet()

# start model training and model logging
train_and_log_model(lenet_model,"LeNet-5",train_dataset,val_dataset,epochs=5)

# **AlexNet**

In [ ]:
def build_alexnet(input_shape=(224,224,3), num_classes=1):
  model= models.Sequential()

  # normalization
  model.add(layers.Rescaling(1./255, input_shape=input_shape))

  # 1st conv layer
  model.add(layers.Conv2D(96,(11,11),strides=4,activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3,3),strides=2))

  # 2nd Convolutional Layer
  model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  # 3rd, 4th, and 5th Convolutional Layers
  model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  # Flatten
  model.add(layers.Flatten())

  # 1st Fully Connected Layer
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))

  # 2nd Fully Connected Layer
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))

  # Output Layer
  model.add(layers.Dense(num_classes, activation='sigmoid'))

  return model

In [ ]:
# train and track for alexnet

# call the model
alexnet_model = build_alexnet()

# track and log 
train_and_log_model(alexnet_model,"AlexNet",train_dataset, val_dataset, epochs=5)

# **VGG16**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_vgg16(input_shape=(224, 224, 3), num_classes=2):
 
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Fully Connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile
    loss_fn = 'binary_crossentropy' 
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

    return model


In [ ]:
# train and track for alexnet

# call the model
vgg_model = build_vgg16()

# track and log 
train_and_log_model(vgg_model,"VGG16",train_dataset, val_dataset, epochs=5)

# **VGG16 transfer learning**

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

def build_vgg16_transfer(input_shape=(224,224,3), num_classes=1, freeze_conv=True):

    # Load VGG16 without the top fully-connected layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze convolutional layers if required
    if freeze_conv:
        for layer in base_model.layers:
            layer.trainable = False

    # Build new model on top
    model = models.Sequential()
    model.add(layers.Rescaling(1./255, input_shape=input_shape))  # normalize inputs
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='sigmoid'))  # binary classification

    return model


In [ ]:
# train and track for alexnet

# call the model
vgg_model_transfer = build_vgg16_transfer()

# track and log 
train_and_log_model(vgg_model_transfer,"VGG16_transfer",train_dataset, val_dataset, epochs=5)

# **USING DATA AUGMENTATION, L2 REGULARIZATION AND DROPOUTS for robust model**

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import mlflow
import mlflow.keras
import matplotlib.pyplot as plt
import seaborn as sns

# Data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

def train_and_log_model_aug(model_fn, model_name, train_ds, val_ds, epochs=5, optimizer='adam'):

    with mlflow.start_run(run_name=model_name):
        
        # Build model with augmentation & regularization
        input_shape = (224, 224, 3)
        model = tf.keras.Sequential([
            data_augmentation,
            layers.Rescaling(1./255, input_shape=input_shape),
            model_fn(input_shape=input_shape),  # your base model
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid',
                         kernel_regularizer=regularizers.l2(0.01))
        ])
        
        # Compile
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        
        # Train
        history = model.fit(train_ds,
                            validation_data=val_ds,
                            epochs=epochs)
        
        # Log params
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("epochs", epochs)
        
        # Log final metrics
        mlflow.log_metrics({
            "train_accuracy": history.history['accuracy'][-1],
            "val_accuracy": history.history['val_accuracy'][-1],
            "train_loss": history.history['loss'][-1],
            "val_loss": history.history['val_loss'][-1]
        })
        
        # Predictions for confusion matrix
        y_true = np.concatenate([y for x, y in val_ds], axis=0)
        y_pred_probs = model.predict(val_ds, verbose=0)
        y_pred = (y_pred_probs > 0.5).astype(int).flatten()
        
        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Non_Autistic', 'Autistic'],
                    yticklabels=['Non_Autistic', 'Autistic'])
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        
        # Save CM figure
        cm_path = "confusion_matrix.png"
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        
        # Classification report
        report = classification_report(y_true, y_pred, target_names=['Non_Autistic', 'Autistic'], output_dict=True)
        report_path = "classification_report.txt"
        with open(report_path, "w") as f:
            f.write(classification_report(y_true, y_pred, target_names=['Non_Autistic', 'Autistic']))
        mlflow.log_artifact(report_path)
        
        # Log model
        mlflow.keras.log_model(model, "model")
        print(f"✅ Model '{model_name}' trained and logged with MLflow.")
    
    return model


2025-08-12 07:21:24.633568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754983284.817160      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754983284.870826      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'mlflow'

## 1) **LeNet-5**

In [ ]:
# build leNet model but with data augmentation, l2 reg and dropouts
def build_lenet(input_shape=(224, 224, 3)):
    from tensorflow.keras import layers, models
    
    model = models.Sequential()

    # Layer 1
    model.add(layers.Conv2D(6, kernel_size=(5,5), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))

    # Layer 2
    model.add(layers.Conv2D(16, kernel_size=(5,5), activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))

    # Flatten + Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))

    return model

# call model building and tracking
train_and_log_model_aug(build_lenet, "LeNet-5_aug",train_dataset,val_dataset,epochs=5)

## 2) **AlexNet**

In [ ]:
def build_alexnet(input_shape=(224,224,3), num_classes=2):
  model= models.Sequential()

  # normalization
  model.add(layers.Rescaling(1./255, input_shape=input_shape))

  # 1st conv layer
  model.add(layers.Conv2D(96,(11,11),strides=4,activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3,3),strides=2))

  # 2nd Convolutional Layer
  model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  # 3rd, 4th, and 5th Convolutional Layers
  model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  # Flatten
  model.add(layers.Flatten())

  # 1st Fully Connected Layer
  model.add(layers.Dense(4096, activation='relu'))

  # 2nd Fully Connected Layer
  model.add(layers.Dense(4096, activation='relu'))

  # Output Layer
  model.add(layers.Dense(num_classes, activation='sigmoid'))

  return model

train_and_log_model_aug(build_alexnet,"AlexNet_aug",train_dataset,val_dataset,epochs=5)

# Without MLflow

In [3]:
# Load train data
import tensorflow as tf

# define the train directory
import os
path = '/kaggle/input/autism/autism_split'
train_dir = os.path.join(path,'train')
val_dir = os.path.join(path,'val')


train_dataset = tf.keras.utils.image_dataset_from_directory(
train_dir,
labels='inferred',
label_mode='binary', # for binary classification (0/1)
image_size=(224,224),
batch_size=32,
shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
val_dir,
labels='inferred',
label_mode='binary', # for binary classification (0/1)
image_size=(224,224),
batch_size=32,
shuffle=False
)

# Optional: cache + prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
layers.RandomFlip("horizontal"), # mirror flip
layers.RandomRotation(0.1), # small rotations
layers.RandomZoom(0.1), # zoom in/out
layers.RandomTranslation(0.1, 0.1), # shift images
layers.RandomContrast(0.1), # adjust contrast
])



train_dataset = train_dataset.map(
lambda x, y: (data_augmentation(x, training=True), y),
num_parallel_calls=tf.data.AUTOTUNE
)

2025-08-12 13:31:16.154508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755005476.365462      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755005476.435405      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 4670 files belonging to 2 classes.


I0000 00:00:1755005494.565542      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 1168 files belonging to 2 classes.


In [4]:
# Plot training history function (reuse from previous answer)
def plot_training_history(history):
    plt.figure(figsize=(14, 5))

    # Accuracy subplot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss subplot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Evaluate and show classification report and confusion matrix
def evaluate_and_report(model, val_dataset, class_names):
    y_true = []
    y_pred = []
    for images, labels in val_dataset:
        preds = model.predict(images, verbose=0)
        preds_labels = (preds > 0.5).astype(int).flatten()
        y_pred.extend(preds_labels)
        y_true.extend(labels.numpy() if hasattr(labels, 'numpy') else labels)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()


## **LeNet-5**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Your LeNet definition
def build_lenet(input_shape=(224,224,3)):
    model = models.Sequential()
    model.add(layers.Rescaling(1./255, input_shape=input_shape))
    model.add(layers.Conv2D(6, kernel_size=(5,5), activation='relu', padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(16, kernel_size=(5,5), activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model for binary classification
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    
    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )

    return plot_training_history(history),evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

build_lenet()

## **AlexNet**

In [ ]:
from tensorflow.keras.optimizers import Adam

def build_alexnet(input_shape=(224,224,3)):
  model= models.Sequential()

  # normalization
  model.add(layers.Rescaling(1./255, input_shape=input_shape))

  # 1st conv layer
  model.add(layers.Conv2D(96,(11,11),strides=4,activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3,3),strides=2))

  # 2nd Convolutional Layer
  model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  # 3rd, 4th, and 5th Convolutional Layers
  model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  # Flatten
  model.add(layers.Flatten())

  # 1st Fully Connected Layer
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))

  # 2nd Fully Connected Layer
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))

  # Output Layer
  model.add(layers.Dense(1, activation='sigmoid'))

  # Compile model for binary classification
  model.compile(
        optimizer= Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
  # Assuming train_dataset and val_dataset are prepared as in your code before
    
  # Train the model
  history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )

  return plot_training_history(history),evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

# call and build
build_alexnet()

## **VGG-16 :- Transfer Learning**

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
def build_vgg16_transfer(input_shape=(224, 224, 3)):
    
    # Load pretrained VGG16 without top layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))

    # Compile model for binary classification
    model.compile(
        optimizer= Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )
    
    return plot_training_history(history),evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

# call and build
build_vgg16_transfer()


## **VGG16 :- Fine Tuning**

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
# Fine-tuning is done at a very low learning rate (1e-5 or 1e-6) to avoid destroying pretrained weights
from tensorflow.keras.optimizers import Adam

def build_vgg16_finetune(input_shape=(224,224,3), fine_tune_at=None):

    # load the base model
    base_model = VGG16(
        weights = 'imagenet',
        include_top = False,
        input_shape = input_shape
    )

    # Freeze all layers initially
    base_model.trainable = True if fine_tune_at is not None else False
    if fine_tune_at is not None:
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

    # build a new head
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(84,activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile model for binary classification
    model.compile(
        optimizer= Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )
    
    return plot_training_history(history),evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

# call and build
build_vgg16_finetune(fine_tune_at=15)    

## **MobileNet**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

def build_mobilenet_finetune(input_shape=(224,224,3), fine_tune_at=None):
    base_model = tf.keras.applications.MobileNet(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True if fine_tune_at is not None else False
    if fine_tune_at is not None:
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )
    return plot_training_history(history), evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

In [ ]:
#call for fine-tuning MobileNet, unfreeze from layer 80
build_mobilenet_finetune(fine_tune_at=80)

## **ResNet50**

In [ ]:
def build_resnet50_finetune(input_shape=(224,224,3), fine_tune_at=None):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True if fine_tune_at is not None else False
    if fine_tune_at is not None:
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )
    return plot_training_history(history), evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

In [ ]:
#  call for fine-tuning ResNet50, unfreeze from layer 140
build_resnet50_finetune(fine_tune_at=140)

## **InceptionV3**

In [ ]:
def build_inceptionv3_finetune(input_shape=(224,224,3), fine_tune_at=None):
    base_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True if fine_tune_at is not None else False
    if fine_tune_at is not None:
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )
    return plot_training_history(history), evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

In [ ]:
# call for fine-tuning InceptionV3, unfreeze from layer 250
build_inceptionv3_finetune(fine_tune_at=250)

# **OPTIMIZING THE BEST MODEL**: ResNet50

In [5]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
import seaborn as sns
import tempfile
import os


### 1) Training on a larger epoch size (epoch =30)

In [ ]:
def build_resnet50_finetune(input_shape=(224,224,3), fine_tune_at=None):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True if fine_tune_at is not None else False
    if fine_tune_at is not None:
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=30
    )
    return plot_training_history(history), evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

#  call for fine-tuning ResNet50, unfreeze from layer 140
build_resnet50_finetune(fine_tune_at=140)

### 2) Fine-Tuning data, model and HP

Fewer epochs + early stopping stops overfit creep.

L2 reg + dropout controls weight growth.

Class weights slightly bias toward autistic recall.

LR scheduler helps converge smoothly without overshooting.

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_resnet50_finetune(input_shape=(224,224,3), fine_tune_at=140):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    class_weights = {0: 1.2, 1: 1.0}  # Slight autistic recall boost

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=20,
        callbacks=callbacks,
        class_weight=class_weights
    )
    
    return plot_training_history(history), evaluate_and_report(model, val_dataset, class_names=['non-autistic', 'autistic'])

build_resnet50_finetune(fine_tune_at=140)


In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_resnet50_finetune(input_shape=(224,224,3), fine_tune_at=140):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.6),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    class_weights = {0: 1.2, 1: 1.0}  # Slight autistic recall boost

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=30,
        callbacks=callbacks,
        class_weight=class_weights
    )
    
    return plot_training_history(history), evaluate_and_report(model, val_dataset, class_names=['autistic', 'non-autistic'])

build_resnet50_finetune(fine_tune_at=120)


In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Assuming train_dataset and val_dataset are already defined
# e.g. via image_dataset_from_directory

def build_resnet50_finetune(input_shape=(224,224,3), fine_tune_at=140):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    class_weights = {0: 1.2, 1: 1.0}  # Slight autistic recall boost

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=20,
        callbacks=callbacks,
        class_weight=class_weights
    )

    # Save model for FastAPI
    model.save("model/resnet50_autism.h5")
    print("✅ Model saved at model/resnet50_autism.h5")

    return history

# Example call
history = build_resnet50_finetune(fine_tune_at=140)


In [6]:
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_resnet50_finetune(input_shape=(224,224,3), fine_tune_at=140):
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(84, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.6),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    class_weights = {0: 1.2, 1: 1.0}  # Slight autistic recall boost

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=30,
        callbacks=callbacks,
        class_weight=class_weights
    )

    # Save model for FastAPI
    model.save_weights("/kaggle/working/resnet50_autism_weights.weights.h5")
    print("✅ Model saved at model/resnet50_autism.h5")
    
    return model

# build_resnet50_finetune(fine_tune_at=120)


# Model optimization using OPTUNA for Hyperparameter tuning

In [7]:
!pip install optuna

In [8]:
import optuna
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def objective(trial):
    # Search space
    dense_units = trial.suggest_categorical("dense_units", [64, 84, 128, 256])
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.7)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    fine_tune_at = trial.suggest_int("fine_tune_at", 100, 150)
    cw_autistic = trial.suggest_float("cw_autistic", 0.8, 1.5)
    cw_non_autistic = trial.suggest_float("cw_non_autistic", 0.8, 1.5)

    # Model
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224,224,3)
    )
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    class_weights = {0: cw_non_autistic, 1: cw_autistic}

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=15,  # Lower for tuning speed
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=0
    )

    val_acc = max(history.history['val_accuracy'])

    # Save best model weights for current trial
    # model.save_weights(f"/kaggle/working/model_trial_{trial.number}.weights.h5")
    # Save best model for current trial
    model.save(f"/kaggle/working/model_trial_{trial.number}.keras")  # TensorFlow SavedModel format


    return val_acc


In [9]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # Try 15 runs

print("Best trial:", study.best_trial.params)


[I 2025-08-12 07:28:39,297] A new study created in memory with name: no-name-a7dea6ae-bcc7-4a06-bfa8-57ca57bf344f


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


I0000 00:00:1754983752.768679     103 service.cc:148] XLA service 0x7abbf4002ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754983752.769472     103 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754983755.415972     103 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1754983762.290281     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
[I 2025-08-12 07:36:19,364] Trial 0 finished with value: 0.9366438388824463 and parameters: {'dense_units': 256, 'dropout_rate': 0.4235363847214342, 'l2_reg': 0.00011482185941072048, 'learning_rate': 9.879824852833632e-05, 'fine_tune_at': 146, 'cw_autistic': 1.255434787703455, 'cw_non_autistic': 1.327191344277702}. Best is trial 0 with value: 0.9366438388824463.
[I 2025-08-12 07:44:28,241] Trial 1 finished with value: 0.8090753555297852 and parameters: {'dense_

Best trial: {'dense_units': 128, 'dropout_rate': 0.4187845256515822, 'l2_reg': 0.000497250308351491, 'learning_rate': 7.417948021074986e-05, 'fine_tune_at': 118, 'cw_autistic': 1.233759421780937, 'cw_non_autistic': 0.9700421364904332}


### Best params, trial 2 and 7 from optuna hyparameter tuning

[I 2025-08-12 07:52:44,511] **Trial 2** finished with value: **0.9443492889404297**

**Parameters**: {'dense_units': 256, 'dropout_rate': 0.32792881699035276, 'l2_reg': 0.00044048224110103157, 'learning_rate': 1.717631257431285e-05, 'fine_tune_at': 102, 'cw_autistic': 1.1667754741390135, 'cw_non_autistic': 0.9043860829232542}. 


[I 2025-08-12 08:33:22,923] **Trial 7** finished with value: **0.9452054500579834** 

**Parameters**: {'dense_units': 128, 'dropout_rate': 0.4187845256515822, 'l2_reg': 0.000497250308351491, 'learning_rate': 7.417948021074986e-05, 'fine_tune_at': 118, 'cw_autistic': 1.233759421780937, 'cw_non_autistic': 0.9700421364904332}. 

## **Running the best model using the best hyperparameters (trial 7's)**

In [12]:
# best_params = study.best_trial.params
from tensorflow.keras import Input, Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Explicit input tensor
input_tensor = Input(shape=(224, 224, 3))
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=input_tensor
)
base_model.trainable = True
for layer in base_model.layers[:118]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.000497250308351491)),
    layers.Dropout(0.42),
    layers.Dense(1, activation='sigmoid')
])

final_model.compile(
    optimizer=Adam(learning_rate=7e-05),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

final_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

# final_class_weights = {0: best_params['cw_autistic'], 1: best_params['cw_non_autistic']}
final_class_weights = {0: 1.2, 1: 0.97}

final_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=final_callbacks,
    class_weight=final_class_weights
)

final_model.save("/kaggle/working/resnet50_autism_best_model3.keras")
print("✅ Best tuned model saved for FastAPI.")



TypeError: A symbolic KerasTensor cannot be used as a boolean.

In [13]:
from tensorflow.keras import Input, Model, regularizers, layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Explicit input tensor
input_tensor = Input(shape=(224, 224, 3))
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_tensor=input_tensor  # correct usage
)
base_model.trainable = True
for layer in base_model.layers[:118]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(0.000497250308351491)),
    layers.Dropout(0.42),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=7e-05),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

final_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

final_class_weights = {0: 1.2, 1: 0.97}

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=final_callbacks,
    class_weight=final_class_weights
)

model.save("/kaggle/working/resnet50_autism_best_model3.keras")
print("✅ Best tuned model saved for FastAPI.")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_8']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)
I0000 00:00:1755002666.299625     105 service.cc:148] XLA service 0x7a9ba0003010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755002666.300443     105 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1755002666.300460     105 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1755002669.292347     105 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1755002681.937868     105 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


146/146 ━━━━━━━━━━━━━━━━━━━━ 99s 359ms/step - accuracy: 0.6974 - loss: 0.7378 - val_accuracy: 0.8202 - val_loss: 0.5390 - learning_rate: 7.0000e-05
Epoch 2/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 33s 221ms/step - accuracy: 0.8458 - loss: 0.4856 - val_accuracy: 0.8647 - val_loss: 0.4408 - learning_rate: 7.0000e-05
Epoch 3/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 33s 219ms/step - accuracy: 0.8882 - loss: 0.3850 - val_accuracy: 0.8818 - val_loss: 0.4613 - learning_rate: 7.0000e-05
Epoch 4/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 33s 219ms/step - accuracy: 0.9283 - loss: 0.3132 - val_accuracy: 0.8913 - val_loss: 0.3917 - learning_rate: 7.0000e-05
Epoch 5/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 33s 222ms/step - accuracy: 0.9363 - loss: 0.2771 - val_accuracy: 0.8870 - val_loss: 0.4303 - learning_rate: 7.0000e-05
Epoch 6/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 33s 219ms/step - accuracy: 0.9495 - loss: 0.2462 - val_accuracy: 0.8622 - val_loss: 0.7152 - learning_rate: 7.0000e-05
Epoch 7/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 33s 222ms/step - 

In [7]:
import tensorflow as tf
from tensorflow.keras import Input, Model, regularizers, layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define input tensor
input_tensor = Input(shape=(224, 224, 3))

# Load the ResNet50 base model with pretrained ImageNet weights and without top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_tensor=input_tensor
)

# Freeze the layers up to the specified layer for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:118]:
    layer.trainable = False

# Add custom head on top of the base model
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(
    128,
    activation='relu',
    kernel_regularizer=regularizers.l2(0.000497250308351491)
)(x)
x = layers.Dropout(0.42)(x)
output = layers.Dense(1, activation='sigmoid')(x)

# Build the final model
model = Model(inputs=input_tensor, outputs=output)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=7e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Prepare callbacks
final_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

# Use class weights as per your setup
final_class_weights = {0: 1.2, 1: 0.97}

# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=final_callbacks,
    class_weight=final_class_weights
)

# Save the full model for later loading (in FastAPI, etc.)
model.save("/kaggle/working/resnet50_autism_best_model4.keras")
print("✅ Best tuned model saved for FastAPI.")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30


I0000 00:00:1755005551.036968     102 service.cc:148] XLA service 0x7af484002890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755005551.037613     102 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1755005554.125732     102 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/146 ━━━━━━━━━━━━━━━━━━━━ 12s 88ms/step - accuracy: 0.7109 - loss: 0.7741   

I0000 00:00:1755005564.119135     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


146/146 ━━━━━━━━━━━━━━━━━━━━ 91s 314ms/step - accuracy: 0.7195 - loss: 0.7019 - val_accuracy: 0.7962 - val_loss: 0.5349 - learning_rate: 7.0000e-05
Epoch 2/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 29s 193ms/step - accuracy: 0.8523 - loss: 0.4853 - val_accuracy: 0.8031 - val_loss: 0.6708 - learning_rate: 7.0000e-05
Epoch 3/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 29s 195ms/step - accuracy: 0.8924 - loss: 0.3890 - val_accuracy: 0.8142 - val_loss: 0.6808 - learning_rate: 7.0000e-05
Epoch 4/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 30s 200ms/step - accuracy: 0.9217 - loss: 0.3353 - val_accuracy: 0.8767 - val_loss: 0.4545 - learning_rate: 7.0000e-05
Epoch 5/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 29s 196ms/step - accuracy: 0.9412 - loss: 0.2781 - val_accuracy: 0.9033 - val_loss: 0.3875 - learning_rate: 7.0000e-05
Epoch 6/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 29s 196ms/step - accuracy: 0.9674 - loss: 0.2197 - val_accuracy: 0.9127 - val_loss: 0.3823 - learning_rate: 7.0000e-05
Epoch 7/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 29s 196ms/step - 

In [54]:
from tensorflow import keras

model_path = "/kaggle/working/resnet50_autism_best_model2.keras"
model = keras.models.load_model(model_path)

# Test loading
print(model.summary())


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_11     │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 84)             │       172,116 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,061,629 (225.30 MB)

 Trainable params: 17,650,857 (67.33 MB)

 Non-trainable params: 6,109,056 (23.30 MB)

 Optimizer params: 35,301,716 (134.67 MB)

None


In [75]:
import numpy as np
from tensorflow.keras.preprocessing import image

def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)  # Load image
    img_array = image.img_to_array(img)                      # Convert to array
    img_array = np.expand_dims(img_array, axis=0)            # Add batch dimension
    # img_array /= 255.0                                       # Normalize (if trained that way)
    return img_array


In [76]:
# function for making predictions

def pred_aut(img_path):
    
    # Preprocess
    processed_img = preprocess_image(img_path)
    
    # Predict
    pred = model.predict(processed_img)
    
    # Interpret output
    # predicted_class = np.argmax(pred, axis=1)
    # print(f"Predicted Class: {predicted_class}")
    print(f"Pred probability: {pred}")
    

    return "Autistic" if pred < 0.5 else "Not Autistic"


In [103]:
pred_aut("/kaggle/input/real-test/image (7).jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Pred probability: [[1.9573296e-05]]


'Autistic'

# Testing

In [102]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# ----------------------------
# 1. Load the model
# ----------------------------
model_path = "/kaggle/working/resnet50_autism_best_model2.keras"  # Update if needed
model = load_model(model_path)

# ----------------------------
# 2. Test data setup
# ----------------------------
test_dir = "/kaggle/input/autism/autism_split/test"  # Directory with 'autistic' & 'non_autistic' folders

img_height, img_width = 224, 224  # Change if your model expects different size
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# ----------------------------
# 3. Predictions
# ----------------------------
pred_probs = model.predict(test_generator)
pred_classes = (pred_probs > 0.5).astype(int).ravel()

# ----------------------------
# 4. Metrics
# ----------------------------
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

print("\nClassification Report:")
print(classification_report(true_classes, pred_classes, target_names=class_labels))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, pred_classes))


Found 24 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

Classification Report:
              precision    recall  f1-score   support

    autistic       0.50      1.00      0.67        12
non_autistic       0.00      0.00      0.00        12

    accuracy                           0.50        24
   macro avg       0.25      0.50      0.33        24
weighted avg       0.25      0.50      0.33        24


Confusion Matrix:
[[12  0]
 [12  0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# resave the model

In [7]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/resnet50_autism_best_model2.keras", compile=False)
model.save("/kaggle/working/resnet50_autism_safe.keras", save_format="keras")


ValueError: File not found: filepath=/kaggle/working/resnet50_autism_best_model2.keras. Please ensure the file is an accessible `.keras` zip file.